In [ ]:
"""
This script generates the subplots for figure 8
These are generated using the timeseries outputs from the HGTM
"""

In [30]:
import netCDF4 as nc
import numpy as np
from matplotlib import pyplot as plt

In [31]:
diameter_nc_ffn = '/home/meso/Insync/onedrive/sync/papers/hail-xsec_paper/hgtm-output/growth_timeseries.nc'
regime_nc_ffn = '/home/meso/Insync/onedrive/sync/papers/hail-xsec_paper/hgtm-output/growth_regime.nc'
output_file_folder = '/home/meso/Insync/onedrive/sync/papers/hail-xsec_paper/hgtm-output/xsecs_images'

In [32]:
#read required datasets
with nc.Dataset(diameter_nc_ffn) as ncid:
    diameters = ncid['GROWTH_TIMESERIES_ALL'][:] #diameters (in mm)
with nc.Dataset(regime_nc_ffn) as ncid:
    regimes = ncid['GROWTH_REGIME_ALL'][:] #regime, where 1 is wet growth and 0 is dry growth

In [42]:
ignore_list = [2,10,11] #reduce number of samples to match observation (select to preserve bin count)
n_hail = np.shape(diameters)[1]

In [46]:
#loop for each hailstone
for hail_idx in range(n_hail):
    #skip hailstone if in ignore list
    if hail_idx in ignore_list:
        continue
    
    #extract dataset
    hail_diameter_ts = diameters[:,hail_idx]
    hail_regime_ts = regime[:,hail_idx]
    nan_mask = np.isnan(hail_diameter_ts)
    hail_diameter_ts = hail_diameter_ts[~nan_mask]/2
    hail_regime_ts = hail_regime_ts[~nan_mask]
    
    
    # set up figure
    fig = plt.figure(figsize=[10, 10])
    fig.set_facecolor("k")
    ax = plt.subplot(111)
    ax.set_aspect("equal")
    ax.set_facecolor("k")
    
    # draw entire hail
    max_radius = np.max(hail_diameter_ts)
    background = plt.Circle((0, 0), max_radius, color=[0.5, 0.5, 0.5])
    ax.add_artist(background)
    
    #for each timestep
    for i in range(len(hail_diameter_ts)):
        #add layers in reverse
        if hail_regime_ts[-i] == +1:
            #wet growth
            layer = plt.Circle((0, 0), hail_diameter_ts[-i], color=[0.5, 0.5, 0.5])
        else:
            #dry growth
            layer = plt.Circle((0, 0), hail_diameter_ts[-i], color=[1.0, 1.0, 1.0])
        ax.add_artist(layer)
    
    # draw embryo
    embryo = plt.Circle((0, 0), hail_diameter_ts[0], color=[1.0, 1.0, 1.0])
    ax.add_artist(embryo)
    
    # draw grids on plot
    grid_coord = [-35, -25, -15, -5, 5, 15, 25, 35]
    for i in grid_coord:
        plt.plot([i, i], [-40, 40], "y:", lw=2)
        plt.plot([-40, 40], [i, i], "y:", lw=2)
    ax.set_xlim((-40, 40))
    ax.set_ylim((-40, 40))
    
    # export plot
    ax.set_axis_off()
    plt.savefig(
        f"{output_file_folder}/hgtm_eq_{hail_idx+1:02}.png",
        bbox_inches="tight",
        pad_inches=0,
    )
    plt.clf()
    plt.close("all")